In [1]:
#librerias estandar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [2]:
data = pd.read_csv(r"C:\Users\USER\Desktop\intro_ML\train.csv", index_col="ID")
data.drop(columns=["F_TIENEINTERNET.1"], inplace=True)  
data.head()

,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
ID,,,,,,,,,,,,,,,,,,,
904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


Nota: Se me recomendó este modelo para intentar conseguir accuracys más altos, así que empezaré a implementarlo para ver su desempeño. Aparentemente para este modelo no es necesario preprocesar los datos porque este lo hace ya internamente, sin embargo probaré de ambas formas.

In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report
from catboost import CatBoostClassifier
import numpy as np

#Separamiento
y = data["RENDIMIENTO_GLOBAL"]
X = data.drop(columns=["RENDIMIENTO_GLOBAL"])

#Pasos de preprocesamiento (identificar tipos de variables)
cat_features = [col for col in X.columns if X[col].dtype == "object"]

#Nans

for col in cat_features:
    X[col] = X[col].astype(str).fillna("DESCONOCIDO").replace("nan", "DESCONOCIDO")

#No mas preprocesamiento

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

#modelo

model_base = CatBoostClassifier(
    loss_function="MultiClass",
    eval_metric="Accuracy",
    verbose=False,
    task_type="GPU",   
)

#Iteración para encontrar los mejores hiperparámetros

param_dist = {
    "iterations": [1500, 2500, 3500, 5000],
    "learning_rate": [0.01, 0.02, 0.03, 0.05],
    "depth": [6, 8, 10, 12],
    "l2_leaf_reg": [1, 3, 5, 7, 9],
    "random_strength": [1, 2, 3, 5],
    "bagging_temperature": [0.1, 0.3, 0.5, 1.0],
    "border_count": [32, 64, 128, 254],
    "auto_class_weights": ["Balanced", None]
}

#busqueda aleatoria

search = RandomizedSearchCV(
    estimator=model_base,
    param_distributions=param_dist,
    n_iter=40,                # puedes subir a 40 para aún mejor resultado
    scoring="accuracy",
    cv=3,
    random_state=42,
    verbose=2
)

search.fit(
    X_train, y_train,
    cat_features=cat_features
)

print("\nMEJORES HIPERPARÁMETROS ENCONTRADOS:\n", search.best_params_)


best_model = search.best_estimator_

best_model.fit(
    X_train, y_train,
    cat_features=cat_features,
    verbose=200
)



y_pred = best_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("\n=====================================")
print("ACCURACY FINAL:", acc)
print("=====================================\n")

print("Classification Report:")
print(classification_report(y_test, y_pred))


Tamaño original: (692500, 19)
Columnas categóricas detectadas: ['E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO', 'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA', 'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD', 'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_EDUCACIONMADRE']
Fitting 3 folds for each of 40 candidates, totalling 120 fits
[CV] END auto_class_weights=Balanced, bagging_temperature=1.0, border_count=32, depth=8, iterations=2500, l2_leaf_reg=5, learning_rate=0.01, random_strength=5; total time= 1.2min
[CV] END auto_class_weights=Balanced, bagging_temperature=1.0, border_count=32, depth=8, iterations=2500, l2_leaf_reg=5, learning_rate=0.01, random_strength=5; total time= 1.2min
[CV] END auto_class_weights=Balanced, bagging_temperature=1.0, border_count=32, depth=8, iterations=2500, l2_leaf_reg=5, learning_rate=0.01, random_strength=5; total time= 1.3min
[CV] END auto_class_weights=Balanced, bagging_temper

c:\Users\USER\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
36 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
36 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\USER\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\USER\AppData\Local\Programs\Python\Python313\Lib\site-packages\catboost\core.py", line 5245, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, graph, sa


MEJORES HIPERPARÁMETROS ENCONTRADOS:
 {'random_strength': 2, 'learning_rate': 0.02, 'l2_leaf_reg': 1, 'iterations': 2500, 'depth': 8, 'border_count': 64, 'bagging_temperature': 0.1, 'auto_class_weights': 'Balanced'}
0:	learn: 0.3744947	total: 175ms	remaining: 7m 18s
200:	learn: 0.4151103	total: 9.26s	remaining: 1m 45s
400:	learn: 0.4244942	total: 17.5s	remaining: 1m 31s
600:	learn: 0.4295368	total: 24.5s	remaining: 1m 17s
800:	learn: 0.4340592	total: 32.9s	remaining: 1m 9s
1000:	learn: 0.4381382	total: 41.3s	remaining: 1m 1s
1200:	learn: 0.4415604	total: 49.9s	remaining: 53.9s
1400:	learn: 0.4448267	total: 58.1s	remaining: 45.6s
1600:	learn: 0.4474943	total: 1m 6s	remaining: 37.5s
1800:	learn: 0.4498955	total: 1m 15s	remaining: 29.2s
2000:	learn: 0.4523316	total: 1m 23s	remaining: 20.9s
2200:	learn: 0.4547765	total: 1m 32s	remaining: 12.5s
2400:	learn: 0.4568647	total: 1m 40s	remaining: 4.15s
2499:	learn: 0.4581770	total: 1m 44s	remaining: 0us

ACCURACY FINAL: 0.4329025270758123

Clas

In [ ]:
#modelo con mejores parametros

y = data["RENDIMIENTO_GLOBAL"]                 # variable a predecir
X = data.drop(columns=["RENDIMIENTO_GLOBAL"])  # todo lo demás

cat_features = X.select_dtypes(include=['object']).columns.tolist()

print("Columnas categóricas detectadas:")
print(cat_features)


for col in cat_features:
    X[col] = X[col].astype(str).fillna("missing")


X = X.fillna(-1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42,
    stratify=y
)

#modelo encontrado
best_params = {
    'random_strength': 2,
    'learning_rate': 0.02,
    'l2_leaf_reg': 1,
    'iterations': 2500,
    'depth': 8,
    'border_count': 64,
    'bagging_temperature': 0.1,
    'auto_class_weights': 'Balanced'
}

model = CatBoostClassifier(
    **best_params,
    task_type="GPU",         
    loss_function="MultiClass",
    eval_metric="Accuracy",
    random_seed=42,
    verbose=200
)

model.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    use_best_model=True,
    cat_features=cat_features
)

y_pred = model.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print("Accuracy:", acc)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Columnas categóricas detectadas:
['E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO', 'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA', 'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD', 'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_EDUCACIONMADRE']
0:	learn: 0.3758079	test: 0.3744487	best: 0.3744487 (0)	total: 160ms	remaining: 6m 40s
200:	learn: 0.4148237	test: 0.4112199	best: 0.4112199 (200)	total: 11.9s	remaining: 2m 16s
400:	learn: 0.4246848	test: 0.4206856	best: 0.4206856 (400)	total: 42s	remaining: 3m 40s
600:	learn: 0.4295346	test: 0.4248194	best: 0.4249866 (585)	total: 1m	remaining: 3m 10s
800:	learn: 0.4341234	test: 0.4274505	best: 0.4275155 (798)	total: 1m 10s	remaining: 2m 29s
1000:	learn: 0.4375402	test: 0.4290895	best: 0.4291412 (999)	total: 1m 21s	remaining: 2m 1s
1200:	learn: 0.4405390	test: 0.4297760	best: 0.4298347 (1198)	total: 1m 31s	remaining: 1m 39s
1400:	learn: 0.4438318	test: 0.4307308	best: 0.4

In [ ]:
from catboost import Pool
#Prueba con datos de test del kaggle
X_test = pd.read_csv(r"C:\Users\USER\Desktop\intro_ML\test.csv", index_col="ID")

#preprocesamiento similar al de entrenamiento
cat_features = model.get_cat_feature_indices()


columnas_modelo = model.feature_names_
X_test = X_test[columnas_modelo]

for col in columnas_modelo:
    if X_test[col].dtype == "object":
        X_test[col] = X_test[col].astype(str).fillna("missing")

test_pool = Pool(X_test, cat_features=cat_features)


y_pred = model.predict(test_pool)

output = pd.DataFrame({
    "ID": X_test.index,
    "RENDIMIENTO_GLOBAL": y_pred[:,0]
})

output.to_csv("submission_catboost.csv", index=False)


Cat features indices: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
Listo. Archivo creado.


In [25]:
y_pred[:,0].shape

(296786,)